In [33]:
# Pkg.add("Interact")
# Pkg.add("DataStructures")

using Tokamak
using DataStructures
using Interact
using Unitful
using SymPy

In [34]:
function parse_tokamak_data(solved_case)
    cur_table_data = OrderedDict(
        "eta_CD" => @sprintf("%.3g", solved_case["eta_CD"]),
        "B_0" => @sprintf("%.4g", solved_case["B_0"]),
        "P_F" => @sprintf("%.5g", solved_case["P_F"]),
        "I_M" => @sprintf("%.3g", solved_case["I_M"]),
        "R_0" => @sprintf("%.3g", solved_case["R_0"]),
        " a " => @sprintf("%.3g", solved_case["a"]),
        "q_K" => @sprintf("%.3g", solved_case["q_star"]),
        "T_i" => @sprintf("%.3g", solved_case["T_k"]),
        "n_e" => @sprintf("%.3g", solved_case["n_bar"]),
        "f_CD" => @sprintf("%.3g", solved_case["f_CD"]),
        "f_B" => @sprintf("%.3g", solved_case["f_B"]),
        "P_H" => @sprintf("%.3g", solved_case["P_H"]),
        "beta_N" => @sprintf("%.4g", solved_case["beta_N"] * Tokamak.max_beta_N),
        "P_W" => @sprintf("%.4g", solved_case["P_W"] * Tokamak.max_P_W),
        "P_alpha" => @sprintf("%.4g", solved_case["P_alpha"]),
        "P_BR" => @sprintf("%.4g", solved_case["P_BR"]),
        "q_P" => @sprintf("%.4g", solved_case["q_P"]),
        "rho_j" => @sprintf("%.4g", solved_case["rho_j"]),
        "Q" => @sprintf("%.4g", solved_case["Q"]),
        "kappa" => @sprintf("%.4g", solved_case["kappa"]),
        "delta" => @sprintf("%.4g", solved_case["delta"]),
        "epsilon" => @sprintf("%.4g", solved_case["epsilon"]),
        "N_G" => @sprintf("%.4g", solved_case["N_G"]),
        "H" => @sprintf("%.4g", solved_case["H"]),
        "nu_n" => @sprintf("%.4g", solved_case["nu_n"]),
        "nu_T" => @sprintf("%.4g", solved_case["nu_T"]),
        "Z_eff" => @sprintf("%.4g", solved_case["Z_eff"]),
        "rho_m" => @sprintf("%.4g", solved_case["rho_m"]),
        "f_DT" => @sprintf("%.4g", solved_case["f_DT"]),
        "wave_theta" => @sprintf("%.4g", solved_case["wave_theta"])
    ) 
    
    cur_table_data
end

parse_tokamak_data (generic function with 1 method)

In [35]:
function arc_data()
    cur_table_data = OrderedDict()
    
    cur_table_data["main"] = OrderedDict(
        "eta_CD" => 0.321,
        "B_0" => 9.2,
        "P_F" => 525,
        "I_M" => 7.8,
        "R_0" => 3.3,
        " a " => 1.13,
        "q_K" => 4.7,
        "T_i" => 14,
        "n_e" => 1.3,
        "f_CD" => 0.37,
        "f_B" => 0.63,
        "P_H" => 39,
        "beta_N" => 0.0259,
        "P_W" => 2.5,
        "P_alpha" => "-",
        "P_BR" => "-",
        "q_P" => "-",
        "rho_j" => "-",
        "Q" => 13.6,
        "kappa" => 1.84,
        "delta" => "-",
        "epsilon" => 0.342,
        "N_G" => 0.67,
        "H" => 1.8,
        "nu_n" => 0.385,
        "nu_T" => 0.929,
        "Z_eff" => 1.2,
        "rho_m" => "-",
        "f_DT" => "-",
        "wave_theta" => "-"        
    ) 

    cur_table_data
end

arc_data (generic function with 1 method)

In [36]:
function act_1_data()
    cur_table_data = OrderedDict()
    
    cur_table_data["main"] = OrderedDict(
        "eta_CD" => 0.188,
        "B_0" => 6.0,
        "P_F" => 1813,
        "I_M" => 10.95,
        "R_0" => 6.25,
        " a " => 1.56,
        "q_K" => 4.5,
        "T_i" => 20.6,
        "n_e" => 1.3,
        "f_CD" => 0.09,
        "f_B" => 0.91,
        "P_H" => 42.7,
        "beta_N" => 0.056,
        "P_W" => 2.45,
        "P_alpha" => 363,
        "P_BR" => 56.3,
        "q_P" => 13.7,
        "rho_j" => "-",
        "Q" => 42.5,
        "kappa" => 1.8,
        "delta" => 0.63,
        "epsilon" => 0.25,
        "N_G" => 1.0,
        "H" => 1.65,
        "nu_n" => 0.27,
        "nu_T" => 1.15,
        "Z_eff" => 2.11,
        "rho_m" => "-",
        "f_DT" => 0.75,
        "wave_theta" => "-"
    ) 

    cur_table_data
end

act_1_data (generic function with 1 method)

In [37]:
function act_2_data()
    cur_table_data = OrderedDict()
    
    cur_table_data["main"] = OrderedDict(
        "eta_CD" => 0.256,
        "B_0" => 8.75,
        "P_F" => 2637,
        "I_M" => 13.98,
        "R_0" => 9.75,
        " a " => 2.44,
        "q_K" => 8.0,
        "T_i" => 17.8,
        "n_e" => 0.86,
        "f_CD" => 0.23,
        "f_B" => 0.77,
        "P_H" => 105.5,
        "beta_N" => 0.026,
        "P_W" => 1.46,
        "P_alpha" => 528,
        "P_BR" => 96.5,
        "q_P" => 10.0,
        "rho_j" => "-",
        "Q" => 25.0,
        "kappa" => 2.1,
        "delta" => 0.575,
        "epsilon" => 0.25,
        "N_G" => 1.3,
        "H" => 1.22,
        "nu_n" => 0.41,
        "nu_T" => 1.15,
        "Z_eff" => 2.12,
        "rho_m" => "-",
        "f_DT" => 0.74,
        "wave_theta" => "-"        
    ) 

    cur_table_data
end

act_2_data (generic function with 1 method)

In [38]:
B_0_dict = OrderedDict(
    "act_1" => 6.0,
    "act_2" => 8.75,
    "arc" => 9.2
)

T_k_dict = OrderedDict(
    "act_1" => 20.6,
    "act_2" => 17.8,
    "arc" => 14.0
)

nu_n_dict = OrderedDict(
    "act_1" => 0.27,
    "act_2" => 0.41,
    "arc" => 0.385
)

vars_dict = OrderedDict()

vars_dict["act_1"] = OrderedDict(
    "R_0" => [6.25],
    "T_k" => [20.6],
    "rho_j" => [0.0],
    "eta_CD" => [0.188],
    "B_0" => [6.0],
    "success" => true,
    "beta_N" => [0.056],
    "P_W" => [2.5],
    "n_bar" => [1.3],
    "I_M" => [10.95],
    "f_B" => [0.91],
    "q_star" => [4.5],
    "P_F" => [1813]
)

vars_dict["act_2"] = OrderedDict(
    "R_0" => [9.75],
    "T_k" => [17.8],
    "rho_j" => [0.0],
    "eta_CD" => [0.256], 
    "B_0" => [8.75],   
    "nu_n" => [0.41],
    "success" => true,
    "beta_N" => [0.026],
    "P_W" => [1.46],
    "n_bar" => [0.86],
    "I_M" => [13.98],
    "f_B" => [0.77],
    "q_star" => [8],
    "P_F" => [2637]
)

vars_dict["arc"] = OrderedDict(
    "R_0" => [3.3],
    "T_k" => [14],
    "rho_j" => [0.0],
    "eta_CD" => [0.321],
    "B_0" => [9.2],
    "nu_n" => [0.385],
    "success" => true,
    "beta_N" => [.0259],
    "P_W" => [2.5],
    "n_bar" => [1.3],
    "I_M" => [7.8],
    "f_B" => [0.63],
    "q_star" => [7.2],
    "P_F" => [525]
)


vars_dict

DataStructures.OrderedDict{Any,Any} with 3 entries:
  "act_1" => DataStructures.OrderedDict{String,Any}("R_0"=>[6.25],"T_k"=>[20.6]…
  "act_2" => DataStructures.OrderedDict{String,Any}("R_0"=>[9.75],"T_k"=>[17.8]…
  "arc"   => DataStructures.OrderedDict{String,Any}("R_0"=>[3.3],"T_k"=>[14],"r…

In [39]:
param_dict = Dict(
    "nu_n" => linspace(round(0.41-0.25,3),round(0.41+0.25,3),21),
    "nu_T" => linspace(round(1.15-0.25,3),round(1.15+0.25,3),11),
    "f_DT" => 0.5:0.05:1.0,
    "BR_scaling" => linspace(-0.5,2.5,13),
    "enable_eta_CD_derive" => [false, true],
    "limit" => ["auto", "beta", "wall"],
    "default_eta_CD" => linspace(0.1,0.3,9),
    "rho_m" => linspace(0.3,0.9,13),
    "Z_eff" => linspace(round(2.12-0.72,2),round(2.12+0.72),9),
    "wave_theta" => 0:30.0:360,
    "Q" => 15:2.5:45
)

other_keys = ["none"]
append!(other_keys, [ cur_key for cur_key in keys(param_dict)] )

default_vals = Dict()

for (cur_key, cur_val) in param_dict
    if cur_key == "wave_theta"
        default_vals[cur_key] = 60
    elseif isa(cur_val, Array)
        default_vals[cur_key] = first(cur_val)
    else
        default_vals[cur_key] = mean(cur_val)
    end
end

default_vals

Dict{Any,Any} with 11 entries:
  "limit"                => "auto"
  "Q"                    => 30.0
  "nu_n"                 => 0.41
  "f_DT"                 => 0.75
  "Z_eff"                => 2.2
  "nu_T"                 => 1.15
  "default_eta_CD"       => 0.2
  "BR_scaling"           => 1.0
  "enable_eta_CD_derive" => false
  "rho_m"                => 0.6
  "wave_theta"           => 60

In [40]:
function make_output_table(solved_cases, design, column)
    
    tokamak_jl_data = OrderedDict()
    
    for (cur_key, cur_val) in solved_cases
        tokamak_jl_data[cur_key] = parse_tokamak_data(cur_val)
    end
        
    if design == "Act 1"
        paper_data = act_1_data()
    elseif design == "Act 2"
        paper_data = act_2_data()
    elseif design == "Arc"
        paper_data = arc_data()   
    else
        error("Invalid design for paper selection")
    end
    
    header_row = ["Variable", "Paper"]
 
    for cur_key in keys(tokamak_jl_data)
        push!(header_row, cur_key)
    end

    num_keys = length(keys(paper_data[column]))
    
    mapped_data = [ [] for ii in 1:num_keys ]
    
    for (cur_index, cur_key) in enumerate(keys(paper_data[column]))
        push!(mapped_data[cur_index], cur_key)
        
        push!(mapped_data[cur_index], paper_data[column][cur_key])
        
        for cur_val in values(tokamak_jl_data)
            cur_class = nothing
            
            try
                parsed_val = parse(cur_val[cur_key])
                rel_error = abs( paper_data[column][cur_key] - parsed_val )
                rel_error /= min( paper_data[column][cur_key], parsed_val )
                
                if rel_error > 0.2
                    cur_class = "danger"
                elseif rel_error > 0.1
                    cur_class = "warning"
                elseif rel_error < 1e-5 || ( ( cur_key == "P_W" || cur_key == "beta_N" ) && rel_error < 1.5*Tokamak.wave_error_level )
                    cur_class = "info"
                elseif rel_error < 0.02
                    cur_class = "success"
                else
                    cur_class = ""
                end
            catch
                cur_class = ""
            end
            
            if cur_class == ""
                push!(mapped_data[cur_index], cur_val[cur_key])
            else
                push!(mapped_data[cur_index], 
                    Dict(
                        "value" => cur_val[cur_key],
                        "class" => cur_class
                    )
                )                
            end
        end       
    end
    
    cur_table = vcat( (reshape(x, (1, :)) for x in mapped_data)... )

    Tokamak.HTMLElements.table(cur_table, header_row=header_row, title="$design Input")    
end

make_output_table (generic function with 1 method)

In [41]:
@manipulate for 
    design=["arc", "act_2", "act_1"], 
    column=["main"], 
    limit=param_dict["limit"], 
    eta_CD=param_dict["enable_eta_CD_derive"],#,
    delta in slider(0.3:0.025:0.63, value=0.4, label="delta"),
    rho_m=slider(0.3:0.05:0.8, value=0.7, label="rho_m"),
    N_G=slider(0.5:0.025:1.5, value=1.0, label="N_G"),
#     Q in slider(param_dict["Q"], value=42.5, label="Q"), 
    f_DT=slider(0.75:0.05:1.0, value=0.9, label="f_DT"),
#     BR=param_dict["BR_scaling"], 
    nu_n=slider(50:5:150, value=100, label="nu_n [%]"), 
#     nu_T=param_dict["nu_T"], 
#     rho_m=slider(0.6:0.05:0.8, value=0.7, label="rho_m"), 
    kappa in slider(linspace(1.8,2.2,9), value=1.8, label="kappa"), 
#     delta in slider(0.575:0.025:0.65, value=0.63, label="delta"), 
#     Z_eff=slider(param_dict["Z_eff"], value=2.11, label="Z_eff"), 
#     CD_guess=param_dict["default_eta_CD"], 
    wave_theta in slider(0:15:180,value=150, label="theta"),
#     other_key=other_keys, 
#     other_val="", 
    ready=[false, true]
    
    if !ready
        return
    end    
    
    solved_cases = OrderedDict()
    
#     if other_key == "none" || other_val == ""
        key_list = ["Standard", "Piped Data"]
#     else
#         if other_key == "f_DT"
#             first_val = f_DT
#         elseif other_key == "BR_scaling"
#             first_val = BR
#         elseif other_key == "rho_m"
#             first_val = rho_m
#         elseif other_key == "enable_eta_CD_derive"
#             first_val = eta_CD
#         elseif other_key == "nu_n"
#             first_val = nu_n
#         elseif other_key == "nu_T"
#             first_val = nu_T            
#         elseif other_key == "limit"
#             first_val = limit
#         elseif other_key == "Z_eff"
#             first_val = Z_eff            
#         elseif other_key == "Q"
#             first_val = Q                        
#         elseif other_key == "wave_theta"
#             first_val = wave_theta                        
#         end
        
#         key_list = ["$other_key = $first_val"]
        
#         if other_val == ""
#             other_val = default_vals[other_key]
#         end
        
#         push!(key_list, "$other_key = $other_val")        
#     end
    
    Tokamak.load_input("defaults.jl", true)
    Tokamak.load_input("$(design).jl", true)  

    for cur_key in key_list
        println(321)
        Tokamak.load_input("defaults.jl", true)
        Tokamak.load_input("$(design).jl", true)

        Tokamak.load_input(" enable_eta_CD_derive = $eta_CD ")

#         Tokamak.load_input(" BR_scaling = $BR ")
        Tokamak.load_input(" nu_n = $nu_n * $(nu_n_dict[design]) / 100 ")
#         Tokamak.load_input(" nu_T = $nu_T ")

        Tokamak.load_input(" kappa = $kappa ")
        Tokamak.load_input(" delta = $delta ")
        
        Tokamak.load_input(" rho_m = $rho_m ")
        
        Tokamak.load_input(" f_DT = $f_DT ")
        Tokamak.load_input(" N_G = $N_G ")
# #         Tokamak.load_input(" Z_eff = $Z_eff ")
        
# #         Tokamak.load_input(" default_eta_CD = $CD_guess ")
        
#         Tokamak.load_input(" wave_theta = $wave_theta ")
        
# #         Tokamak.load_input(" Q = $Q ")
        
#         if other_val != "" && cur_key != "Tokamak.jl"
#             if other_key != "f_DT" && other_key != "limit"
#                 Tokamak.load_input(cur_key)
#             end
#         end
        
     
#         if other_val != "" && cur_key != "Tokamak.jl"
#             if other_key == "f_DT"
#                 cur_f_DT = parse(split(cur_key,"=")[2])
#             elseif other_key == "limit"
#                 cur_limit = strip(split(cur_key,"=")[2])
#             end    
#         end
        
#         cur_f_DT = f_DT

#         if design == "act_2" && isapprox(cur_f_DT, 0.75)
#             cur_f_DT = 0.74
#         end

#         Tokamak.load_input(" f_DT = $cur_f_DT ")
        
#         cur_limit = limit
        
#         if cur_limit != "auto"
#             Tokamak.load_input(" main_constraint = \"$(cur_limit)\"")
#         end

        if cur_key == "Piped Data"
            found_data = vars_dict[design]
        else
            found_data = Tokamak.sweep_T_k([T_k_dict[design]])
        end

        filter!((k,v) -> !isa(v, OrderedDict), found_data)

        for (cur_sub_key, cur_sub_val) in found_data
            found_data[cur_sub_key] = first(cur_sub_val)
        end
        
        solved_cases[cur_key] = Tokamak.analyze_solved_case(found_data)
    end
    
    table_html = make_output_table(solved_cases, replace(ucfirst(design), "_", " "), column)    
    
    IJulia.clear_output(true)
    display("text/html", table_html)
end

return

Variable,Paper,Standard,Piped Data
eta_CD,0.321,0.321,0.321
B_0,9.2,0.6952,9.2
P_F,525,430.21,8863.4
I_M,7.8,20.8,20.8
R_0,3.3,68,3.3
a,1.13,23.3,1.13
q_K,4.7,2.82,1.81
T_i,14,14,14
n_e,1.3,0.0122,5.19
f_CD,0.37,0.587,0.587


In [42]:
# using Tokamak
# using Unitful
# using SymPy
    
function custom_subs(cur_func, design)
  subs(
    cur_func,
    Tokamak.symbol_dict["beta_N"] => Tokamak.max_beta_N,
    Tokamak.symbol_dict["P_W"] => Tokamak.max_P_W,
    Tokamak.symbol_dict["tau_factor"] => Tokamak.max_tau_factor,
    Tokamak.symbol_dict["T_k"] => vars_dict[design]["T_k"][1],
    Tokamak.symbol_dict["n_bar"] => vars_dict[design]["n_bar"][1],
    Tokamak.symbol_dict["B_0"] => vars_dict[design]["B_0"][1],
    Tokamak.symbol_dict["I_M"] => vars_dict[design]["I_M"][1],
  )
end

function custom_solve(cur_func, design)
  solve(
    custom_subs(cur_func, design)
  )[1]
end

function R_list(design)
  cur_list = OrderedDict()

  cur_list["paper"] = vars_dict[design]["R_0"][1]

#   cur_list["beta"] = custom_subs(SymPy.solve( Tokamak.R_B_2() - Tokamak.K_2() * Tokamak.G_2() , Tokamak.symbol_dict["beta_N"] )[1], design)

  cur_list["f_B"] = custom_solve( vars_dict[design]["f_B"][1] - Tokamak.f_B() , design )

  cur_list["P_F"] = custom_solve(
    vars_dict[design]["P_F"][1] - 
    Tokamak.calc_possible_values(
      custom_subs(
        Tokamak.P_F()/1u"MW",
                design
      ) / Tokamak.symbol_dict["R_0"]^3,
      cur_eta_CD=Tokamak.default_eta_CD
      ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"
    ) * Tokamak.symbol_dict["R_0"]^3
    , design
  )

  cur_list["eq1"] = custom_solve(
    Tokamak.calc_possible_values(
      custom_subs(
        Tokamak.R_B_1() - Tokamak.K_1() * Tokamak.G_1(),
        design
      ),
      cur_eta_CD=Tokamak.default_eta_CD
      ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"
    ),
    design
  )

  cur_list["eq2"] = custom_solve(
    Tokamak.calc_possible_values(
      custom_subs(
        Tokamak.symbol_dict["R_0"]*Tokamak.symbol_dict["B_0"] - Tokamak.K_2() * Tokamak.G_2(),
        design
      ),
      cur_eta_CD=Tokamak.default_eta_CD
      ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"
    ),
    design
  )

  cur_list["eq3"] = custom_solve(
    Tokamak.calc_possible_values(
      custom_subs(
        Tokamak.symbol_dict["R_0"]^3 - Tokamak.K_3() * Tokamak.G_3(),
        design
      ),
      cur_eta_CD=Tokamak.default_eta_CD
      ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"
    ),
    design
  )

  cur_list["q_star"] = custom_solve( vars_dict[design]["q_star"][1] - Tokamak.q_star() , design )

  cur_list["green"] = custom_solve( 
    Tokamak.symbol_dict["n_bar"]-Tokamak.N_G*Tokamak.symbol_dict["I_M"]/(pi*(Tokamak.a()/1u"m")^2), design 
  )

  cur_list["steady"] = custom_solve( 
    Tokamak.calc_possible_values(
  custom_subs(
    Tokamak.symbol_dict["n_bar"]*Tokamak.symbol_dict["R_0"]^2-Tokamak.K_n()*Tokamak.symbol_dict["T_k"]/Tokamak.calc_possible_values(Tokamak.symbol_dict["K_CD_denom"],
  cur_eta_CD=Tokamak.default_eta_CD
  ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"),
            design
  ),
  cur_eta_CD=Tokamak.default_eta_CD
  ,cur_T_k=vars_dict[design]["T_k"][1]*1u"keV"
),
        design
  )        

  cur_list
end

@manipulate for 
    design=["arc", "act_2", "act_1"],
    Q = 50:5:150,
    kappa = 50:5:150,
    delta = 50:5:150,
    epsilon = 50:5:150,
    N_G = 50:5:150,
    H = 50:5:150,
    nu_n = 50:5:150,
    nu_T = 50:5:150,
    Z_eff = 50:5:150,
    rho_m = 50:5:150,
    f_DT = 50:5:150,
    max_beta_N = 50:5:150,
    max_P_W = 50:5:150
    
    Tokamak.load_input("defaults.jl", true)
    Tokamak.load_input("$(design).jl", true)
    
    println(444)
    Tokamak.load_input(" Q = $(Tokamak.Q) * $Q / 100 ")
    Tokamak.load_input(" kappa = $(Tokamak.kappa) * $kappa / 100 ")
    Tokamak.load_input(" delta = $(Tokamak.delta) * $delta / 100 ")
    Tokamak.load_input(" epsilon = $(Tokamak.epsilon) * $epsilon / 100 ")
    Tokamak.load_input(" N_G = $(Tokamak.N_G) * $N_G / 100 ")
    Tokamak.load_input(" H = $(Tokamak.H) * $H / 100 ")
    Tokamak.load_input(" nu_n = $(Tokamak.nu_n) * $nu_n / 100 ")
    Tokamak.load_input(" nu_T = $(Tokamak.nu_T) * $nu_T / 100 ")
    Tokamak.load_input(" Z_eff = $(Tokamak.Z_eff) * $Z_eff / 100 ")
    println(2222)
    Tokamak.load_input(" rho_m = $(Tokamak.rho_m) * $rho_m / 100 ")
    Tokamak.load_input(" f_DT = $(Tokamak.f_DT) * $f_DT / 100 ")
    Tokamak.load_input(" max_beta_N = $(Tokamak.max_beta_N) * $max_beta_N / 100 ")
    Tokamak.load_input(" max_P_W = $(Tokamak.max_P_W) * $max_P_W / 100 ") 
    println("lol")
    cur_str_array = [" ----- ","  $design"," ----- ",""]
    
    for (cur_key, cur_val) in R_list(design)
        println(cur_key)
        println(cur_val)
        push!(cur_str_array, " $cur_key = $(round(cur_val,4)) ")
        if cur_key == "paper"
           push!(cur_str_array, " ") 
        end
    end   
    println("sd")
    
    push!(cur_str_array, " ")
    println(123)
    push!(cur_str_array, " Q = $(Tokamak.Q) ")
    push!(cur_str_array, " kappa = $(Tokamak.kappa) ")
    push!(cur_str_array, " delta = $(Tokamak.delta) ")
    push!(cur_str_array, " epsilon = $(Tokamak.epsilon) ")
    push!(cur_str_array, " N_G = $(Tokamak.N_G) ")
    push!(cur_str_array, " H = $(Tokamak.H) ")
    push!(cur_str_array, " nu_n = $(Tokamak.nu_n) ")
    push!(cur_str_array, " nu_T = $(Tokamak.nu_T) ")
    println(321)
    push!(cur_str_array, " Z_eff = $(Tokamak.Z_eff) ")
    push!(cur_str_array, " rho_m = $(Tokamak.rho_m) ")
    push!(cur_str_array, " f_DT = $(Tokamak.f_DT) ")
    push!(cur_str_array, " max_beta_N = $(round(Tokamak.max_beta_N,4)) ")
    push!(cur_str_array, " max_P_W = $(Tokamak.max_P_W) ")      
    
    push!(cur_str_array, " R = $(vars_dict[design]["R_0"][1]) ")  
    push!(cur_str_array, " B = $(vars_dict[design]["B_0"][1]) ")  
    push!(cur_str_array, " T = $(vars_dict[design]["T_k"][1]) ")  
    push!(cur_str_array, " eta_CD = $(vars_dict[design]["eta_CD"][1]) ")  
    
    push!(cur_str_array, " ")
    
    IJulia.clear_output(true)
    println(join(cur_str_array,"\n"))    
    
end

return

 ----- 
  arc
 ----- 

 paper = 3.3 
 
 f_B = 3.1295 
 P_F = 3.2149 
 eq1 = 5.3615 
 eq2 = 3.4420 
 eq3 = 3.2921 
 q_star = 4.7601 
 green = 3.3076 
 steady = 3.5892 
 
 Q = 13.6 
 kappa = 1.84 
 delta = 0.4 
 epsilon = 0.342 
 N_G = 0.67 
 H = 1.8 
 nu_n = 0.385 
 nu_T = 0.929 
 Z_eff = 1.2 
 rho_m = 0.6 
 f_DT = 0.9 
 max_beta_N = 0.0259 
 max_P_W = 2.5 
 R = 3.3 
 B = 9.2 
 T = 14 
 eta_CD = 0.321 
 


In [43]:
cur_dict_1 = DataStructures.OrderedDict{Any,Any}("rho_m = 0.7"=>DataStructures.OrderedDict{Any,Any}("f_DT = 0.9"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54291],"T_k"=>[20.4518],"rho_j"=>[0.78211],"eta_CD"=>[0.23916],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.851344],"wall"=>[0.996359])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.33742],"T_k"=>[20.6468],"rho_j"=>[0.808702],"eta_CD"=>[0.213407],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00067],"wall"=>[1.5294]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.62438],"T_k"=>[20.6787],"rho_j"=>[0.787473],"eta_CD"=>[0.235913],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.852658],"wall"=>[0.996414])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.42342],"T_k"=>[20.8911],"rho_j"=>[0.813438],"eta_CD"=>[0.210523],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00078],"wall"=>[1.52342])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.46887],"T_k"=>[20.2374],"rho_j"=>[0.7807],"eta_CD"=>[0.238397],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.849788],"wall"=>[0.996141])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.25757],"T_k"=>[20.4204],"rho_j"=>[0.807753],"eta_CD"=>[0.212352],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00058],"wall"=>[1.53646]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54816],"T_k"=>[20.4574],"rho_j"=>[0.786091],"eta_CD"=>[0.235148],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.851056],"wall"=>[0.996159])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.34093],"T_k"=>[20.6568],"rho_j"=>[0.812512],"eta_CD"=>[0.209459],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00067],"wall"=>[1.53053]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.46756],"T_k"=>[23.2877],"rho_j"=>[0.797105],"eta_CD"=>[0.250432],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.868569],"wall"=>[1.00203])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.42678],"T_k"=>[23.8712],"rho_j"=>[0.819905],"eta_CD"=>[0.228753],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0003],"wall"=>[1.45125]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.58255],"T_k"=>[23.6311],"rho_j"=>[0.802634],"eta_CD"=>[0.247166],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.8708],"wall"=>[1.00326])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54104],"T_k"=>[24.1951],"rho_j"=>[0.824424],"eta_CD"=>[0.225941],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00031],"wall"=>[1.44196])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.3736],"T_k"=>[22.9989],"rho_j"=>[0.795484],"eta_CD"=>[0.24959],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.866399],"wall"=>[1.00098])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.27031],"T_k"=>[23.4999],"rho_j"=>[0.818899],"eta_CD"=>[0.226846],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00337],"wall"=>[1.47064]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.48519],"T_k"=>[23.3291],"rho_j"=>[0.800997],"eta_CD"=>[0.246325],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.868498],"wall"=>[1.00199])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.42628],"T_k"=>[23.8698],"rho_j"=>[0.823299],"eta_CD"=>[0.224639],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0003],"wall"=>[1.45069])))))),"f_DT = 0.85"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3122],"T_k"=>[25.6443],"rho_j"=>[0.810006],"eta_CD"=>[0.255983],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.884339],"wall"=>[0.998198])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.31042],"T_k"=>[26.1956],"rho_j"=>[0.828341],"eta_CD"=>[0.236942],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00055],"wall"=>[1.37772]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.4734],"T_k"=>[26.1164],"rho_j"=>[0.815512],"eta_CD"=>[0.252982],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.886762],"wall"=>[0.998392])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.4935],"T_k"=>[26.715],"rho_j"=>[0.83322],"eta_CD"=>[0.234539],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00071],"wall"=>[1.36776])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.1796],"T_k"=>[25.2475],"rho_j"=>[0.808189],"eta_CD"=>[0.254818],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.881996],"wall"=>[0.997954])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.15966],"T_k"=>[25.7692],"rho_j"=>[0.827],"eta_CD"=>[0.235309],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00047],"wall"=>[1.38741]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.333],"T_k"=>[25.6947],"rho_j"=>[0.813666],"eta_CD"=>[0.251789],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.884297],"wall"=>[0.998097])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.33337],"T_k"=>[26.2608],"rho_j"=>[0.83187],"eta_CD"=>[0.232856],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00057],"wall"=>[1.37766]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.057],"T_k"=>[31.2041],"rho_j"=>[0.830855],"eta_CD"=>[0.271835],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.913878],"wall"=>[1.00729])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.3452],"T_k"=>[32.1493],"rho_j"=>[0.843856],"eta_CD"=>[0.25866],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00063],"wall"=>[1.26395]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.3835],"T_k"=>[32.0973],"rho_j"=>[0.836537],"eta_CD"=>[0.269711],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.915254],"wall"=>[0.998868])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.8427],"T_k"=>[30.5033],"rho_j"=>[0.828336],"eta_CD"=>[0.270302],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.909519],"wall"=>[1.00471])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.0674],"T_k"=>[31.3559],"rho_j"=>[0.841898],"eta_CD"=>[0.256145],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00043],"wall"=>[1.27675]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.1217],"T_k"=>[31.4029],"rho_j"=>[0.834613],"eta_CD"=>[0.267615],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.914791],"wall"=>[1.0082])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.4274],"T_k"=>[32.3859],"rho_j"=>[0.847353],"eta_CD"=>[0.254772],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00075],"wall"=>[1.26182]))))))),"rho_m = 0.66"=>DataStructures.OrderedDict{Any,Any}("f_DT = 0.9"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.25434],"T_k"=>[22.4625],"rho_j"=>[0.791841],"eta_CD"=>[0.248589],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.863162],"wall"=>[0.9996])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.1265],"T_k"=>[22.909],"rho_j"=>[0.815847],"eta_CD"=>[0.225406],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00249],"wall"=>[1.48092]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.35993],"T_k"=>[22.7686],"rho_j"=>[0.797293],"eta_CD"=>[0.245363],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.865056],"wall"=>[1.00031])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.24197],"T_k"=>[23.2471],"rho_j"=>[0.820694],"eta_CD"=>[0.222604],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00304],"wall"=>[1.475])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.16601],"T_k"=>[22.198],"rho_j"=>[0.790304],"eta_CD"=>[0.247757],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.861219],"wall"=>[0.998907])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.02905],"T_k"=>[22.6256],"rho_j"=>[0.81474],"eta_CD"=>[0.224244],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00211],"wall"=>[1.48766]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.2686],"T_k"=>[22.4933],"rho_j"=>[0.795757],"eta_CD"=>[0.24453],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.863016],"wall"=>[0.999486])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.14097],"T_k"=>[22.9514],"rho_j"=>[0.819575],"eta_CD"=>[0.221436],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00257],"wall"=>[1.48163]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3717],"T_k"=>[25.8408],"rho_j"=>[0.807419],"eta_CD"=>[0.260948],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.879774],"wall"=>[0.99772])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.4269],"T_k"=>[26.7102],"rho_j"=>[0.827583],"eta_CD"=>[0.24199],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00052],"wall"=>[1.39007]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.5366],"T_k"=>[26.3238],"rho_j"=>[0.813044],"eta_CD"=>[0.257906],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.882195],"wall"=>[0.997858])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.6181],"T_k"=>[27.2558],"rho_j"=>[0.832552],"eta_CD"=>[0.239572],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00066],"wall"=>[1.37967])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.2485],"T_k"=>[25.4694],"rho_j"=>[0.805658],"eta_CD"=>[0.259899],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.877554],"wall"=>[0.99751])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.28502],"T_k"=>[26.3064],"rho_j"=>[0.826333],"eta_CD"=>[0.240482],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00045],"wall"=>[1.39944]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.4064],"T_k"=>[25.9299],"rho_j"=>[0.811245],"eta_CD"=>[0.256835],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.879869],"wall"=>[0.997612])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.46747],"T_k"=>[26.826],"rho_j"=>[0.831257],"eta_CD"=>[0.238025],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00055],"wall"=>[1.38926])))))),"f_DT = 0.85"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.5682],"T_k"=>[29.3964],"rho_j"=>[0.823579],"eta_CD"=>[0.268642],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.903666],"wall"=>[1.00223])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.7749],"T_k"=>[30.3097],"rho_j"=>[0.83848],"eta_CD"=>[0.253827],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00034],"wall"=>[1.29691]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.8179],"T_k"=>[30.1734],"rho_j"=>[0.829603],"eta_CD"=>[0.265931],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.907955],"wall"=>[1.00425])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.0325],"T_k"=>[31.0368],"rho_j"=>[0.84329],"eta_CD"=>[0.25177],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00043],"wall"=>[1.28256])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.3831],"T_k"=>[28.8176],"rho_j"=>[0.821352],"eta_CD"=>[0.267207],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.90028],"wall"=>[1.00109])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.564],"T_k"=>[29.7949],"rho_j"=>[0.837383],"eta_CD"=>[0.251711],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00299],"wall"=>[1.31866]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.6172],"T_k"=>[29.5359],"rho_j"=>[0.827264],"eta_CD"=>[0.264451],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.904124],"wall"=>[1.00251])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.8263],"T_k"=>[30.4547],"rho_j"=>[0.8419],"eta_CD"=>[0.249754],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00035],"wall"=>[1.29505]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.517],"T_k"=>[39.3448],"rho_j"=>[0.853224],"eta_CD"=>[0.290154],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.957034],"wall"=>[1.03867])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.2618],"T_k"=>[39.2768],"rho_j"=>[0.857259],"eta_CD"=>[0.28226],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.972477],"wall"=>[1.06288])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[13.9641],"T_k"=>[37.1644],"rho_j"=>[0.846983],"eta_CD"=>[0.287429],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.939787],"wall"=>[1.01298])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.5682],"T_k"=>[39.3067],"rho_j"=>[0.855494],"eta_CD"=>[0.285738],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.952749],"wall"=>[1.02324])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))))))))

cur_dict_2 = DataStructures.OrderedDict{Any,Any}("rho_m = 0.7"=>DataStructures.OrderedDict{Any,Any}("f_DT = 0.9"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.4083],"T_k"=>[20.0626],"rho_j"=>[0.779545],"eta_CD"=>[0.237766],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.848513],"wall"=>[0.995985])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.1922],"T_k"=>[20.2354],"rho_j"=>[0.806969],"eta_CD"=>[0.211481],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00053],"wall"=>[1.54234]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.48578],"T_k"=>[20.2768],"rho_j"=>[0.784938],"eta_CD"=>[0.234517],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.849745],"wall"=>[0.995973])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.27345],"T_k"=>[20.4655],"rho_j"=>[0.811728],"eta_CD"=>[0.208582],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0006],"wall"=>[1.53646])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54291],"T_k"=>[20.4518],"rho_j"=>[0.78211],"eta_CD"=>[0.23916],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.851344],"wall"=>[0.996359])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.33742],"T_k"=>[20.6468],"rho_j"=>[0.808702],"eta_CD"=>[0.213407],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00067],"wall"=>[1.5294]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.62438],"T_k"=>[20.6787],"rho_j"=>[0.787473],"eta_CD"=>[0.235913],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.852658],"wall"=>[0.996414])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.42342],"T_k"=>[20.8911],"rho_j"=>[0.813438],"eta_CD"=>[0.210523],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00078],"wall"=>[1.52342])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.46887],"T_k"=>[20.2374],"rho_j"=>[0.7807],"eta_CD"=>[0.238397],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.849788],"wall"=>[0.996141])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.25757],"T_k"=>[20.4204],"rho_j"=>[0.807753],"eta_CD"=>[0.212352],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00058],"wall"=>[1.53646]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54816],"T_k"=>[20.4574],"rho_j"=>[0.786091],"eta_CD"=>[0.235148],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.851056],"wall"=>[0.996159])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.34093],"T_k"=>[20.6568],"rho_j"=>[0.812512],"eta_CD"=>[0.209459],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00067],"wall"=>[1.53053]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.29682],"T_k"=>[22.7647],"rho_j"=>[0.794151],"eta_CD"=>[0.248892],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.864656],"wall"=>[1.00022])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.18519],"T_k"=>[23.2479],"rho_j"=>[0.817931],"eta_CD"=>[0.225864],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00293],"wall"=>[1.47605]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.40567],"T_k"=>[23.0844],"rho_j"=>[0.799651],"eta_CD"=>[0.245629],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.866655],"wall"=>[1.00108])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.35552],"T_k"=>[23.6692],"rho_j"=>[0.822708],"eta_CD"=>[0.223713],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00029],"wall"=>[1.45872])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.46756],"T_k"=>[23.2877],"rho_j"=>[0.797105],"eta_CD"=>[0.250432],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.868569],"wall"=>[1.00203])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.42678],"T_k"=>[23.8712],"rho_j"=>[0.819905],"eta_CD"=>[0.228753],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0003],"wall"=>[1.45125]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.58255],"T_k"=>[23.6311],"rho_j"=>[0.802634],"eta_CD"=>[0.247166],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.8708],"wall"=>[1.00326])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.54104],"T_k"=>[24.1951],"rho_j"=>[0.824424],"eta_CD"=>[0.225941],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00031],"wall"=>[1.44196])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.3736],"T_k"=>[22.9989],"rho_j"=>[0.795484],"eta_CD"=>[0.24959],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.866399],"wall"=>[1.00098])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.27031],"T_k"=>[23.4999],"rho_j"=>[0.818899],"eta_CD"=>[0.226846],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00337],"wall"=>[1.47064]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.48519],"T_k"=>[23.3291],"rho_j"=>[0.800997],"eta_CD"=>[0.246325],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.868498],"wall"=>[1.00199])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.42628],"T_k"=>[23.8698],"rho_j"=>[0.823299],"eta_CD"=>[0.224639],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0003],"wall"=>[1.45069])))))),"f_DT = 0.85"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.0744],"T_k"=>[24.9334],"rho_j"=>[0.806744],"eta_CD"=>[0.253852],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.880121],"wall"=>[0.997808])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.03862],"T_k"=>[25.4274],"rho_j"=>[0.825948],"eta_CD"=>[0.233975],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00041],"wall"=>[1.39539]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.2203],"T_k"=>[25.3574],"rho_j"=>[0.812157],"eta_CD"=>[0.250819],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.882309],"wall"=>[0.997883])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.20492],"T_k"=>[25.8973],"rho_j"=>[0.830748],"eta_CD"=>[0.231489],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00049],"wall"=>[1.38585])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3122],"T_k"=>[25.6443],"rho_j"=>[0.810006],"eta_CD"=>[0.255983],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.884339],"wall"=>[0.998198])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.31042],"T_k"=>[26.1956],"rho_j"=>[0.828341],"eta_CD"=>[0.236942],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00055],"wall"=>[1.37772]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.4734],"T_k"=>[26.1164],"rho_j"=>[0.815512],"eta_CD"=>[0.252982],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.886762],"wall"=>[0.998392])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.4935],"T_k"=>[26.715],"rho_j"=>[0.83322],"eta_CD"=>[0.234539],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00071],"wall"=>[1.36776])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.1796],"T_k"=>[25.2475],"rho_j"=>[0.808189],"eta_CD"=>[0.254818],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.881996],"wall"=>[0.997954])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.15966],"T_k"=>[25.7692],"rho_j"=>[0.827],"eta_CD"=>[0.235309],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00047],"wall"=>[1.38741]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.333],"T_k"=>[25.6947],"rho_j"=>[0.813666],"eta_CD"=>[0.251789],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.884297],"wall"=>[0.998097])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.33337],"T_k"=>[26.2608],"rho_j"=>[0.83187],"eta_CD"=>[0.232856],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00057],"wall"=>[1.37766]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.6734],"T_k"=>[29.9586],"rho_j"=>[0.826334],"eta_CD"=>[0.269057],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.906237],"wall"=>[1.00313])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.905],"T_k"=>[30.8939],"rho_j"=>[0.840782],"eta_CD"=>[0.254566],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00037],"wall"=>[1.28733]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.9367],"T_k"=>[30.793],"rho_j"=>[0.832475],"eta_CD"=>[0.266335],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.910932],"wall"=>[1.00571])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.251],"T_k"=>[31.8802],"rho_j"=>[0.846043],"eta_CD"=>[0.253378],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00057],"wall"=>[1.27355])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.057],"T_k"=>[31.2041],"rho_j"=>[0.830855],"eta_CD"=>[0.271835],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.913878],"wall"=>[1.00729])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.3452],"T_k"=>[32.1493],"rho_j"=>[0.843856],"eta_CD"=>[0.25866],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00063],"wall"=>[1.26395]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.3835],"T_k"=>[32.0973],"rho_j"=>[0.836537],"eta_CD"=>[0.269711],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.915254],"wall"=>[0.998868])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.8427],"T_k"=>[30.5033],"rho_j"=>[0.828336],"eta_CD"=>[0.270302],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.909519],"wall"=>[1.00471])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.0674],"T_k"=>[31.3559],"rho_j"=>[0.841898],"eta_CD"=>[0.256145],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00043],"wall"=>[1.27675]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[12.1217],"T_k"=>[31.4029],"rho_j"=>[0.834613],"eta_CD"=>[0.267615],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.914791],"wall"=>[1.0082])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.4274],"T_k"=>[32.3859],"rho_j"=>[0.847353],"eta_CD"=>[0.254772],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00075],"wall"=>[1.26182]))))))),"rho_m = 0.66"=>DataStructures.OrderedDict{Any,Any}("f_DT = 0.9"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.0938],"T_k"=>[21.9829],"rho_j"=>[0.789027],"eta_CD"=>[0.24707],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.859646],"wall"=>[0.998407])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[7.94954],"T_k"=>[22.3954],"rho_j"=>[0.813832],"eta_CD"=>[0.223285],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00184],"wall"=>[1.49339]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.19392],"T_k"=>[22.2695],"rho_j"=>[0.794474],"eta_CD"=>[0.243843],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.86137],"wall"=>[0.998886])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.0584],"T_k"=>[22.711],"rho_j"=>[0.818656],"eta_CD"=>[0.220471],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00223],"wall"=>[1.48727])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.25434],"T_k"=>[22.4625],"rho_j"=>[0.791841],"eta_CD"=>[0.248589],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.863162],"wall"=>[0.9996])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.1265],"T_k"=>[22.909],"rho_j"=>[0.815847],"eta_CD"=>[0.225406],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00249],"wall"=>[1.48092]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.35993],"T_k"=>[22.7686],"rho_j"=>[0.797293],"eta_CD"=>[0.245363],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.865056],"wall"=>[1.00031])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.24197],"T_k"=>[23.2471],"rho_j"=>[0.820694],"eta_CD"=>[0.222604],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00304],"wall"=>[1.475])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.16601],"T_k"=>[22.198],"rho_j"=>[0.790304],"eta_CD"=>[0.247757],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.861219],"wall"=>[0.998907])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.02905],"T_k"=>[22.6256],"rho_j"=>[0.81474],"eta_CD"=>[0.224244],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00211],"wall"=>[1.48766]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.2686],"T_k"=>[22.4933],"rho_j"=>[0.795757],"eta_CD"=>[0.24453],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.863016],"wall"=>[0.999486])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[8.14097],"T_k"=>[22.9514],"rho_j"=>[0.819575],"eta_CD"=>[0.221436],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00257],"wall"=>[1.48163]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.15],"T_k"=>[25.173],"rho_j"=>[0.80427],"eta_CD"=>[0.259012],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.875763],"wall"=>[0.997325])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.1706],"T_k"=>[25.9811],"rho_j"=>[0.825286],"eta_CD"=>[0.239254],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00041],"wall"=>[1.4072]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3012],"T_k"=>[25.6128],"rho_j"=>[0.809786],"eta_CD"=>[0.255959],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.87798],"wall"=>[0.997413])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.34622],"T_k"=>[26.4806],"rho_j"=>[0.830197],"eta_CD"=>[0.236765],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00048],"wall"=>[1.3972])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3717],"T_k"=>[25.8408],"rho_j"=>[0.807419],"eta_CD"=>[0.260948],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.879774],"wall"=>[0.99772])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.4269],"T_k"=>[26.7102],"rho_j"=>[0.827583],"eta_CD"=>[0.24199],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00052],"wall"=>[1.39007]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.5366],"T_k"=>[26.3238],"rho_j"=>[0.813044],"eta_CD"=>[0.257906],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.882195],"wall"=>[0.997858])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.6181],"T_k"=>[27.2558],"rho_j"=>[0.832552],"eta_CD"=>[0.239572],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00066],"wall"=>[1.37967])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.2485],"T_k"=>[25.4694],"rho_j"=>[0.805658],"eta_CD"=>[0.259899],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.877554],"wall"=>[0.99751])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.28502],"T_k"=>[26.3064],"rho_j"=>[0.826333],"eta_CD"=>[0.240482],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00045],"wall"=>[1.39944]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.4064],"T_k"=>[25.9299],"rho_j"=>[0.811245],"eta_CD"=>[0.256835],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.879869],"wall"=>[0.997612])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[9.46747],"T_k"=>[26.826],"rho_j"=>[0.831257],"eta_CD"=>[0.238025],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00055],"wall"=>[1.38926])))))),"f_DT = 0.85"=>DataStructures.OrderedDict{Any,Any}("nu_n = 0.33"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.2359],"T_k"=>[28.3614],"rho_j"=>[0.819554],"eta_CD"=>[0.266045],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.897639],"wall"=>[1.00037])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.3915],"T_k"=>[29.2884],"rho_j"=>[0.835958],"eta_CD"=>[0.250062],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.0023],"wall"=>[1.32633]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.4584],"T_k"=>[29.0374],"rho_j"=>[0.825394],"eta_CD"=>[0.263252],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.901189],"wall"=>[1.00144])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.6146],"T_k"=>[29.9462],"rho_j"=>[0.840901],"eta_CD"=>[0.24763],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00327],"wall"=>[1.3155])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.5682],"T_k"=>[29.3964],"rho_j"=>[0.823579],"eta_CD"=>[0.268642],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.903666],"wall"=>[1.00223])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.7749],"T_k"=>[30.3097],"rho_j"=>[0.83848],"eta_CD"=>[0.253827],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00034],"wall"=>[1.29691]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.8179],"T_k"=>[30.1734],"rho_j"=>[0.829603],"eta_CD"=>[0.265931],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.907955],"wall"=>[1.00425])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.0325],"T_k"=>[31.0368],"rho_j"=>[0.84329],"eta_CD"=>[0.25177],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00043],"wall"=>[1.28256])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.3831],"T_k"=>[28.8176],"rho_j"=>[0.821352],"eta_CD"=>[0.267207],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.90028],"wall"=>[1.00109])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.564],"T_k"=>[29.7949],"rho_j"=>[0.837383],"eta_CD"=>[0.251711],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00299],"wall"=>[1.31866]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[11.6172],"T_k"=>[29.5359],"rho_j"=>[0.827264],"eta_CD"=>[0.264451],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.904124],"wall"=>[1.00251])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[10.8263],"T_k"=>[30.4547],"rho_j"=>[0.8419],"eta_CD"=>[0.249754],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["beta"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[1.00035],"wall"=>[1.29505]))))),"nu_n = 0.31"=>DataStructures.OrderedDict{Any,Any}("delta = 0.575"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[13.4071],"T_k"=>[35.286],"rho_j"=>[0.841792],"eta_CD"=>[0.283701],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.929361],"wall"=>[0.997799])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.2831],"T_k"=>[38.0449],"rho_j"=>[0.851733],"eta_CD"=>[0.284672],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.940571],"wall"=>[1.00672])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN])))),"delta = 0.63"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.517],"T_k"=>[39.3448],"rho_j"=>[0.853224],"eta_CD"=>[0.290154],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.957034],"wall"=>[1.03867])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.2618],"T_k"=>[39.2768],"rho_j"=>[0.857259],"eta_CD"=>[0.28226],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.972477],"wall"=>[1.06288])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN])))),"delta = 0.6"=>DataStructures.OrderedDict{Any,Any}("wave_theta = 150.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[13.9641],"T_k"=>[37.1644],"rho_j"=>[0.846983],"eta_CD"=>[0.287429],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.939787],"wall"=>[1.01298])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))),"wave_theta = 140.0"=>DataStructures.OrderedDict{Any,Any}("main_constraint = \"wall\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[14.5682],"T_k"=>[39.3067],"rho_j"=>[0.855494],"eta_CD"=>[0.285738],"B_0"=>[6.0],"success"=>Bool[true],"constraint"=>AbstractString["wall"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[0.952749],"wall"=>[1.02324])),"main_constraint = \"beta\""=>DataStructures.OrderedDict{Any,Any}("R_0"=>[NaN],"T_k"=>[NaN],"rho_j"=>[NaN],"eta_CD"=>[NaN],"B_0"=>[6.0],"success"=>Bool[false],"constraint"=>AbstractString["x"],"limits"=>DataStructures.OrderedDict{Any,Any}("beta"=>[NaN],"wall"=>[NaN]))))))))

function merge_dicts(dict1, dict2, merged_dict=nothing)

  if merged_dict == nothing
    merged_dict = deepcopy(dict1)
    dict1 = merged_dict
  end

  keys1 = collect(keys(dict1))
  keys2 = collect(keys(dict2))

  allkeys = unique(vcat(keys1, keys2))

  if "B_0" in allkeys
    return
  end

  is_identical = length(keys1) == length(allkeys)
  is_identical &= length(keys2) == length(allkeys)

  if is_identical
    for cur_key in allkeys
      merge_dicts(dict1[cur_key], dict2[cur_key], merged_dict)
    end
  else
    for cur_key in allkeys
      if cur_key in keys(dict1) && cur_key in keys(dict2)
        continue
      end

      if cur_key in keys(dict2)
        dict1[cur_key] = dict2[cur_key]
      end
    end    
  end

  merged_dict
end

cur_deep_dict = merge_dicts(cur_dict_1, cur_dict_2)

function squashed_keys(deep_dict, shallow_dict=OrderedDict())
  if "B_0" in keys(deep_dict)
    return
  end

  cur_label = strip(split(first(keys(deep_dict)), "=")[1])

  shallow_dict[cur_label] = []

  for (cur_index, cur_key) in enumerate(keys(deep_dict))
    push!(shallow_dict[cur_label], strip(split(cur_key, "=")[2]))
  end

  squashed_keys(first(values(deep_dict)), shallow_dict)

  shallow_dict  
end

cur_shallow_dict = squashed_keys(cur_deep_dict)

DataStructures.OrderedDict{Any,Any} with 6 entries:
  "rho_m"           => Any["0.7", "0.66"]
  "f_DT"            => Any["0.9", "0.85"]
  "nu_n"            => Any["0.33", "0.31"]
  "delta"           => Any["0.63", "0.6", "0.575"]
  "wave_theta"      => Any["150.0", "140.0"]
  "main_constraint" => Any["\"wall\"", "\"beta\""]

In [44]:
cur_score = OrderedDict(
    "A" => 0,
    "B" => 0,
    "C" => 0,
    "D" => 0,
    "F" => 0
)

for cur_key in keys(Tokamak.reactor_tables[cur_reactor])
    expected_value = Tokamak.reactor_tables[cur_reactor][cur_key]
    actual_value = analyzed_case[cur_key]
    
    ( cur_key == "f_CD") && continue
    ( expected_value == "-") && continue
    iszero( expected_value - actual_value ) && continue
 
    if in(cur_key, values(Tokamak.constraint_params))
        (
            cur_key == Tokamak.constraint_params[analyzed_case["primary_constraint"]] ||
            cur_key == Tokamak.constraint_params[analyzed_case["secondary_constraint"]]
        ) && continue
        
        actual_value *= getfield( Tokamak, Symbol("max_$(cur_key)") )
    end
    
    cur_rel_error = abs(expected_value - actual_value)
    cur_rel_error /= min(expected_value, actual_value)

    if cur_rel_error < 0.05
        cur_score["A"] += 1
    elseif cur_rel_error < 0.1
        cur_score["B"] += 1
    elseif cur_rel_error < 0.25
        cur_score["C"] += 1
    elseif cur_rel_error < 0.5
        cur_score["D"] += 1
    else
        cur_score["F"] += 1
    end  
    
end

println(cur_score)

DataStructures.OrderedDict("A"=>0,"B"=>2,"C"=>3,"D"=>4,"F"=>1)


In [10]:
using DataStructures
using JuliaDB

INFO: Precompiling module JuliaDB.
ERROR: LoadError: ArgumentError: Module IndexedTables not found in current path.
Run `Pkg.add("IndexedTables")` to install the IndexedTables package.
Stacktrace:
 [1] _require(::Symbol) at ./loading.jl:428
 [2] require(::Symbol) at ./loading.jl:398
 [3] include_from_node1(::String) at ./loading.jl:569
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /Applications/JuliaPro-0.5.1.1.app/Contents/Resources/pkgs-0.5.1.1/lib/v0.6/JuliaDB/src/JuliaDB.jl, in expression starting on line 4


LoadError: [91mFailed to precompile JuliaDB to /Applications/JuliaPro-0.5.1.1.app/Contents/Resources/pkgs-0.5.1.1/lib/lib/v0.6/JuliaDB.ji.[39m